# Regression phase 9: LASSO (coordinate descent)

We will implement our very own LASSO solver via coordinate descent. 

* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Import graphlab

In [2]:
import graphlab


# Load hotel data

In [3]:
hotels = graphlab.SFrame('NY.csv') # Chicago.csv
hotels['price'] = hotels['price'].astype(float)
hotels['rates'] = hotels['rates'].astype(float)
hotels['zipcode'] = hotels['zipcode'].astype(float)
hotels['guests'] = hotels['guests'].astype(float)

#hotels = hotels[hotels['size'] < 1500] 
hotels = hotels[hotels['price'] > 10]
hotels

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1492847332.log


This non-commercial license of GraphLab Create for academic use is assigned to dchen@albany.edu and will expire on November 05, 2017.


Finished parsing file /Users/Peter/NY.csv

Parsing completed. Parsed 100 lines in 0.203051 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,float,float,str,str,str,str,int,int,str,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/Peter/NY.csv

Parsing completed. Parsed 26382 lines in 0.190104 secs.

name,zone,zipcode,star,rating,rates,checkin,checkout
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Park Lane Hotel,Central Park,10019.0,4.0,4.0,6.0,04/21/2017,04/22/2017
Park Lane Hotel,Central Park,10019.0,4.0,4.0,6.0,04/21/2017,04/22/2017
Park Lane Hotel,Central Park,10019.0,4.0,4.0,6.0,04/21/2017,04/22/2017
The Belvedere Hotel,Broadway - Times Square,10036.0,3.5,4.2,16.0,04/21/2017,04/22/2017


# Import useful functions from previous phase

In [5]:
import numpy as np 

In [6]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 #this is how to add a constant to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the ohters:
    features = ['constant'] +features # this is how we combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [7]:
# Compute the output 
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

# Normalize features

In the hotel dataset, features vary wildly in their relative magnitude: rating is very large overall compared to star, for instance. As a result, weight for rating would be much smaller than weight for star. This is problematic because "small" weights are dropped first as l1_penalty goes up.

To give equal considerations for all features, we need to normalize features, we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let us first consider a small matrix as a test method

In [8]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:


In [9]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division

In [10]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


We are going write a short function called normalize_features(feature_matrix), which normalizes columns of a given feature matrix. The function should return a pair (normalized_features, norms), where the second item contains the norms of original features.

In [11]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
    normalized_features = feature_matrix/norms
    return (normalized_features, norms)

In [12]:
# test the function
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features


We seek to obtain a sparse set of weights by minimizing the LASSO cost function

    SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).

(By convention, we do not include w[0] in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (we would need to implement a method called subgradient descent). Instead, we will use coordinate descent: at each iteration, we will fix all weights but weight i and find the value of weight i that minimizes the objective. That is, we look for
    argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]

where all weights other than w[i] are held to be constant. We will optimize one w[i] at a time, circling through the weights multiple times.

    1.Pick a coordinate i
    2.Compute w[i] that minimizes the cost function SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)
    3.Repeat Steps 1 and 2 for all coordinates, multiple times

We will use cyclical coordinate descent with normalized features, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:

           ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
    w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
           └ (ro[i] - lambda/2)     if ro[i] > lambda/2

where

    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].

Note that we do not regularize the weight of the constant feature (intercept) w[0], so, for this weight, the update is simply:

    w[0] = ro[i]

# Effect of L1 penalty


Let us consider a simple model with 2 features:

In [13]:
simple_features = ['rating', 'star']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(hotels, simple_features, my_output)
output

array([ 220.,  289.,  224., ...,  265.,  300.,  300.])

normalize features:

In [18]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)
print simple_feature_matrix
print "################################################################"
print simple_feature_matrix[:,0]

[[ 0.00630642  0.00651189  0.00491019]
 [ 0.00630642  0.00651189  0.00491019]
 [ 0.00630642  0.00651189  0.00491019]
 ..., 
 [ 0.00630642  0.00605758  0.00572856]
 [ 0.00630642  0.00605758  0.00572856]
 [ 0.00630642  0.00605758  0.00572856]]
################################################################
[ 0.00630642  0.00630642  0.00630642 ...,  0.00630642  0.00630642
  0.00630642]


We assign some random set of initial weights and inspect the values of ro[i]:

In [19]:
weights = np.array([1., 4., 1.])

Use predict_output() to make predictions on this data.

In [20]:
prediction = predict_output(simple_feature_matrix,weights)

Compute the values of ro[i] for each feature in this simple model:

    ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]

Get a Numpy vector for feature_i using:

    simple_feature_matrix[:,i]

In [21]:
#Very Important
#simple_feature_matrix.shape[1] return 3 so the code below just making a 1*3 vector full of 0s
ro = [0 for i in range((simple_feature_matrix.shape)[1])]
for j in range((simple_feature_matrix.shape)[1]):
    ro[j] = (simple_feature_matrix[:,j] *(output - prediction +(weights[j] *simple_feature_matrix[:,j]))).sum()
print ro

[53270.635854038541, 54210.587831761404, 55316.814541582251]


In [22]:
print 2* ro[1],',',2* ro[2]

108421.175664 , 110633.629083


we have ro[0], ro[1], ro[2]

For W1 to be zero, we need ro[1] in [-lambda/2, lambda/2]

We have -lambda/2 <= ro[1] <= lambda/2

This translates to lambda >= -2ro[1] and lambda >= 2ro[1]

For both conditions to be satisfied, lambda >= 2ro[1] = 1.08e5

Similarly for W2. lambda >= 2ro[2] = 1.11e5.

So, w[i] = 0 if lambda >= 2 * abs(ro[i])


In [23]:
#return true if value is within the threshold ranges otherwise False
#Looking for range -1l_penality/2 <= ro <= l1_penalty/2
def in_l1range(value, penalty):
    return((value >= -penalty / 2.) and (value <= penalty / 2.))

In [24]:
for l1_penalty in [1.00e5, 1.09e5, 1.10e5, 1.30e5, 2.00e5]:
    print in_l1range(ro[1],l1_penalty) , in_l1range(ro[2],l1_penalty)

False False
True False
True False
True True
True True


# Single Coordinate Descent Step


Using the formula above, we will implement coordinate descent that minimizes the cost function over a single feature i.

Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.


In [25]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i] *(output - prediction +(weights[i] *feature_matrix[:,i]))).sum()
    # soft Threshholding!!!
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = (ro_i + l1_penalty/2.)
    elif ro_i > l1_penalty/2.:
        new_weight_i = (ro_i - l1_penalty/2.)
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:


In [27]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


# Cyclical coordinate descent

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:

    1.As we loop over features in order and perform coordinate descent, measure how much each coordinate changes.
    2.After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.
    
Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update our weights variable in-place. See following pseudocode for illustration.**

    for i in range(len(weights)):
        old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
        # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
        #     and old values for weight[i], ..., weight[d-1]
        weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

        # use old_weights_i to compute change in coordinate
        ...

In [28]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    #the # of the features
    D = feature_matrix.shape[1]
    old_weights = np.array(initial_weights)
    change = np.array(initial_weights) * 0.0
    converged = False
    
    while not converged:
        
        for i in range(D):
            new_weights = lasso_coordinate_descent_step(i,feature_matrix,output,
                                                        old_weights,l1_penalty) 
            #compute the change
            change[i] = np.abs(new_weights - old_weights[i])
            #assign new weight
            old_weights[i] = new_weights
        max_change = max(change)
        if max_change < tolerance:
            converged = True
    return old_weights

Use rating and star learn the weights on the hotles dataset


In [84]:
simple_features = ['rating', 'star']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 3e3
tolerance = 1.0

First create a normalized version of the feature matrix, normalized_simple_feature_matrix.


In [85]:
(simple_feature_matrix, output) = get_numpy_data(hotels, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Run the implementation of LASSO coordinate descent:

In [86]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
print weights

[ 11626.13157722      0.          42401.57635627]


In [87]:
prediction = predict_output(normalized_simple_feature_matrix,weights)
prediction
RSS = np.dot(output - prediction, output - prediction)
print 'RSS for normalized dataset = ', RSS

RSS for normalized dataset =  532919827.552


# Evaluating LASSO fit with more features


Let us split the sales dataset into training and test sets.

In [73]:
train_data,test_data = hotels.random_split(.8,seed=0)

Let us consider the following set of features.


In [74]:
all_features = ['star',  
                'zipcode',
                'rating',
                'rates',
                'size',
                'guests'
               ]

First, create a normalized feature matrix from the TRAINING data with these features. 


In [89]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data,all_features,my_output)
normalize_features_matirx, norms = normalize_features(feature_matrix)

First, learn the weights with l1_penalty=3e3, on the training data. Initialize weights to all zeros, and set the tolerance = 1. Call resulting weights weights3e3, we will need them later.

In [102]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 3e3
tolerance = 1.0

In [103]:
weights3e3 = lasso_cyclical_coordinate_descent(normalize_features_matirx,output,
                                              initial_weights,l1_penalty,tolerance)
print weights3e3

[ 11482.20095505  17857.62269066      0.              0.              0.
  20589.84229056      0.        ]


In [104]:
feature_list = ['constant'] + all_features
print feature_list

['constant', 'star', 'zipcode', 'rating', 'rates', 'size', 'guests']


In [106]:
non_zero_weight3e3 = dict(zip(feature_list,weights3e3))
#non_zero_weightle7
for k,v in non_zero_weight3e3.iteritems():
    if v != 0:
        print k,v
    else :
        print '0 value coefficient are: ' , k

0 value coefficient are:  rating
star 17857.6226907
0 value coefficient are:  zipcode
0 value coefficient are:  rates
0 value coefficient are:  guests
constant 11482.2009551
size 20589.8422906


Next, learn the weights with l1_penalty=1e5, on the training data. Initialize weights to all zeros, and set the tolerance=1. Call resulting weights weights1e5, we will need them later.

In [114]:
l1_penalty = 1e5
weights1e5 = lasso_cyclical_coordinate_descent(normalize_features_matirx,output,
                                              initial_weights,l1_penalty,tolerance)
feature_list = ['constant'] + all_features
non_zero_weights1e5 = dict(zip(feature_list,weights1e5))
for k,v in non_zero_weights1e5.iteritems():
    if v != 0:
        print k,v
    else :
        print '0 value coefficient are: ' , k

0 value coefficient are:  rating
0 value coefficient are:  star
0 value coefficient are:  zipcode
0 value coefficient are:  rates
0 value coefficient are:  guests
constant 47676.9575697
0 value coefficient are:  size


Finally, learn the weights with l1_penalty=1e4, on the training data. Initialize weights to all zeros, and set the tolerance=5e5. Call resulting weights weights1e4, we will need them later. 

In [145]:
l1_penalty = 1e3
tolerance = 5e4
weights1e3 = lasso_cyclical_coordinate_descent(normalize_features_matirx,output,
                                              initial_weights,l1_penalty,tolerance)
feature_list = ['constant'] + all_features
non_zero_weights1e3 = dict(zip(feature_list,weights1e3))
for k,v in non_zero_weights1e3.iteritems():
    if v != 0:
        print k,v
    else :
        print 'The zero-value coefficient are: ' , k

rating 43.2950529995
star 2191.22103331
zipcode -1661.62574629
rates 59.8644278169
guests -2641.87427611
constant 47676.9575697
size 4593.05050074


# Rescaling learned weights


To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data:

In this case, we must scale the resulting weights so that we can make predictions with original features:

    1.Store the norms of the original features to a vector called norms:
        features, norms = normalize_features(features)
    2.Run Lasso on the normalized features and obtain a weights vector
    3.Compute the weights for the original features by performing element-wise division, i.e.
        weights_normalized = weights / norms

Now, we can apply weights_normalized to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (weights1e3, weights1e5, weights3e3).

In [156]:
my_output = 'price'
(feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
normalized_feature_matrix, norms = normalize_features(feature_matrix)

normalized_weights1e3 = weights1e3 / norms
normalized_weights3e3 = weights3e3 / norms
normalized_weights1e5 = weights1e5 / norms
print normalized_weights3e3[1]

32.6121477456


In [160]:
print '##########################################'
print 'normalized weights: with L1 = 1e3 '
for i in range(0,7):
    print normalized_weights1e3[i]
print '##########################################'
print 'normalized weights: with L1 = 3e3 '
for i in range(0,7):
    print normalized_weights3e3[i]
print '##########################################'
print 'normalized weights: with L1 = 1e5 '
for i in range(0,7):
    print normalized_weights1e5[i]
print '##########################################'

##########################################
normalized weights: with L1 = 1e3 
335.428995694
4.00167622082
-0.00115306833756
0.0731373921154
0.00122055522058
0.0902066385819
-6.01156124396
##########################################
normalized weights: with L1 = 3e3 
80.782485524
32.6121477456
0.0
0.0
0.0
0.404380587948
0.0
##########################################
normalized weights: with L1 = 1e5 
335.428995694
0.0
0.0
0.0
0.0
0.0
0.0
##########################################


# Evaluating each of the learned models on the test data


Let's now evaluate the three models on the test data:


In [161]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) test_feature_matrix:


In [162]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e3)
RSS = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e7 = ', RSS

RSS for model with weights1e7 =  133256591.788


In [163]:
prediction =  predict_output(test_feature_matrix, normalized_weights3e3)
RSS = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e7 = ', RSS

RSS for model with weights1e7 =  91671642.8756


In [164]:
prediction =  predict_output(test_feature_matrix, normalized_weights1e5)
RSS = np.dot(test_output-prediction, test_output-prediction)
print 'RSS for model with weights1e7 = ', RSS

RSS for model with weights1e7 =  144953135.526


Obviously, the second model performed best on test data. The one with the L1 = 3e3